<H1>Neighbourhoods in Toronto</H1>

<h2>Part I</h2>

Lets import required packages

In [260]:
import pandas as pd
import geocoder
from geopy.geocoders import Nominatim
import folium
import requests
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Read html page into panda

In [126]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = pd.read_html(url)

In [127]:
df = page[0]
df.shape

(180, 3)

In [128]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Remove Not assigned Borough rows

In [129]:
df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)
df.shape

(103, 3)

Check any duplicate Postal Codes

In [130]:
len(df['Postal Code'].unique()) != len(df['Postal Code'])

False

There are no any duplicate Postal Code rows

In [131]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Check any not assigned Neighborhoods in data frame

In [132]:
df["Neighborhood"].isna().sum()

0

There are no Neighborhoods which are not assigned or NaN

In [133]:
#Check Shape of Dataframe
df.shape

(103, 3)

#========================================================#

<h2>Part II</h2>

Lets define a mothod to find Geo Cordinates

In [137]:
def get_geocodination(postal_code):
    
    lat_lng_coords = None
    
    g = geocoder.arcgis('{}, Toronto, Canada'.format(postal_code))
    lat_lng_coords = g.latlng
    
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
            
    return latitude,longitude

Get Latitude and Logitude by calling above funtion and add them to dataframe

In [139]:
df['Latitude'],df['Longitude'] = zip(*df['Postal Code'].apply(get_geocodination))

In [142]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


In [143]:
#Checking any Null values in dataframe
df.isna().sum()

Postal Code     0
Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

In [144]:
df.shape

(103, 5)

Thats the end of part II

#========================================================#

<h2>Part III</h2>

Lets get cordinats in Toronto

In [196]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinates of Toronto, Canada are 43.6534817, -79.3839347.


Creating a map of Toronto with Postal Codes

In [170]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, postalcode in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood'], df['Postal Code']):
    label = '{}, {}, {}'.format(neighborhood, borough, postalcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3ded3d',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Filter Neighborhoods/Postal Codes in boroughs that contains the word Toronto 

In [169]:
Toronto_data = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
Toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
4,M4E,East Toronto,The Beaches,43.678148,-79.295349


Lets visualize postal codes in Boroughs contains the word Toronto (Toronto_data)

In [195]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood, postalcode in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighborhood'], Toronto_data['Postal Code']):
    label = '{}, {}, {}'.format(neighborhood, borough, postalcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3fedfd',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Define Foursquare Credentiols and version

In [199]:
# @hidden_cell

#--hiddden Cell--

LIMIT = 100

Let's create a function to get top 100 venues within 500m for all the neighborhoods/postal codes in Toronto_data

In [207]:
def getNearbyVenues(postalcode, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for postalcode, lat, lng in zip(postalcode, latitudes, longitudes):
        print(postalcode)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postalcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Lets run above funtion in each postal code in Toronto_data

In [209]:
Toronto_venues = getNearbyVenues(postalcode=Toronto_data['Postal Code'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                )

M5A
M7A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


In [210]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1580, 7)


,Postal Code,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,M5A,43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue
2,M5A,43.650964,-79.353041,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
3,M5A,43.650964,-79.353041,BATLgrounds,43.647088,-79.351306,Athletics & Sports
4,M5A,43.650964,-79.353041,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant


Let's check how many venues were returned for each postal code

In [211]:
Toronto_venues.groupby('Postal Code').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M4E,5,5,5,5,5,5
M4K,5,5,5,5,5,5
M4L,21,21,21,21,21,21
M4M,4,4,4,4,4,4
M4N,2,2,2,2,2,2
M4P,6,6,6,6,6,6
M4R,4,4,4,4,4,4
M4S,26,26,26,26,26,26
M4T,5,5,5,5,5,5


Let's find out how many unique categories can be curated from all the returned venues

In [212]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 221 uniques categories.


Analyse each postal code / neighborhood

In [213]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Postal Code column back to dataframe
Toronto_onehot['Postal Code'] = Toronto_venues['Postal Code'] 

# move Postal Code column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Postal Code,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [216]:
Toronto_grouped = Toronto_onehot.groupby('Postal Code').mean().reset_index()
Toronto_grouped.head()

,Postal Code,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M4N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


First, let's write a function to sort venues in descending order.

In [217]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each postal code / neighborhood.

In [256]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = Toronto_grouped['Postal Code']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Pub,Trail,Coffee Shop,Neighborhood,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1,M4K,Bus Line,Business Service,Park,Grocery Store,Discount Store,Yoga Studio,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
2,M4L,Sandwich Place,Fast Food Restaurant,Park,Gym,Italian Restaurant,Pet Store,Pizza Place,Pub,Restaurant,Movie Theater
3,M4M,Night Market,Business Service,Government Building,Baseball Field,Yoga Studio,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
4,M4N,Swim School,Bus Line,Yoga Studio,Food & Drink Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


## Clustering

Run *k*-means to cluster the neighborhood into 5 clusters.

In [257]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:45] 

array([0, 0, 0, 0, 2, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [258]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Postal Code'), on='Postal Code')

Toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,0,Pub,Café,Trail,Athletics & Sports,Mediterranean Restaurant,Theater,Coffee Shop,Distribution Center,French Restaurant,Bank
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,0,Coffee Shop,Café,Sushi Restaurant,Yoga Studio,Discount Store,Sandwich Place,Pharmacy,Park,Middle Eastern Restaurant,Italian Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,0,Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Italian Restaurant,Cosmetics Shop,Bar,Café,Hotel,Theater
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,0,Coffee Shop,Café,Cocktail Bar,American Restaurant,Cosmetics Shop,Gastropub,Seafood Restaurant,Italian Restaurant,Department Store,Creperie
4,M4E,East Toronto,The Beaches,43.678148,-79.295349,0,Health Food Store,Pub,Trail,Coffee Shop,Neighborhood,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
5,M5E,Downtown Toronto,Berczy Park,43.645196,-79.373855,0,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Bakery,Pub,Seafood Restaurant,Restaurant,Breakfast Spot,Hotel
6,M5G,Downtown Toronto,Central Bay Street,43.656072,-79.385653,0,Coffee Shop,Café,Clothing Store,Sandwich Place,Plaza,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Bookstore,Department Store
7,M6G,Downtown Toronto,Christie,43.668602,-79.420387,0,Grocery Store,Café,Baby Store,Playground,Coffee Shop,Park,Candy Store,Yoga Studio,Eastern European Restaurant,Electronics Store
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650542,-79.384116,0,Coffee Shop,Café,Clothing Store,Restaurant,Hotel,Thai Restaurant,Sushi Restaurant,Salad Place,Gym,Deli / Bodega
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.664910,-79.438664,0,Furniture / Home Store,Park,Café,Bakery,Pharmacy,Gym,Grocery Store,Smoke Shop,Pool,Bus Line


Finally, let's visualize the resulting clusters

In [263]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, postalcode in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels'], Toronto_merged['Postal Code']):
    label = folium.Popup(str(poi) + ' (' + str(postalcode) + ')' + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [274]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,0,Pub,Café,Trail,Athletics & Sports,Mediterranean Restaurant,Theater,Coffee Shop,Distribution Center,French Restaurant,Bank
1,M7A,0,Coffee Shop,Café,Sushi Restaurant,Yoga Studio,Discount Store,Sandwich Place,Pharmacy,Park,Middle Eastern Restaurant,Italian Restaurant
2,M5B,0,Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Italian Restaurant,Cosmetics Shop,Bar,Café,Hotel,Theater
3,M5C,0,Coffee Shop,Café,Cocktail Bar,American Restaurant,Cosmetics Shop,Gastropub,Seafood Restaurant,Italian Restaurant,Department Store,Creperie
4,M4E,0,Health Food Store,Pub,Trail,Coffee Shop,Neighborhood,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
5,M5E,0,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Bakery,Pub,Seafood Restaurant,Restaurant,Breakfast Spot,Hotel
6,M5G,0,Coffee Shop,Café,Clothing Store,Sandwich Place,Plaza,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Bookstore,Department Store
7,M6G,0,Grocery Store,Café,Baby Store,Playground,Coffee Shop,Park,Candy Store,Yoga Studio,Eastern European Restaurant,Electronics Store
8,M5H,0,Coffee Shop,Café,Clothing Store,Restaurant,Hotel,Thai Restaurant,Sushi Restaurant,Salad Place,Gym,Deli / Bodega
9,M6H,0,Furniture / Home Store,Park,Café,Bakery,Pharmacy,Gym,Grocery Store,Smoke Shop,Pool,Bus Line


#### Cluster 2

In [273]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,M5P,1,Gym / Fitness Center,Yoga Studio,Dog Run,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Ethiopian Restaurant


#### Cluster 3

In [272]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,M4N,2,Swim School,Bus Line,Yoga Studio,Food & Drink Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


#### Cluster 4

In [271]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M5N,3,Home Service,Spa,Yoga Studio,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Ethiopian Restaurant


#### Cluster 5

In [275]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Postal Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,M4R,4,Playground,Gym Pool,Park,Garden,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
33,M4W,4,Playground,Grocery Store,Park,Candy Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
